$\displaystyle \mathbf{PROCEDIMIENTO ~ ANALÍTICO ~ PARA ~ ENCONTRAR~ LOS~ VALORES ~ DE~~} \alpha  \mathbf{~~Y~~} \beta \\ \mathbf{~~EN~EL~ALGORITMO~ALPHA-MAX~~PLUS~~BETA-MIN}     \tag*{}$

$ \mathbf{Luis~Jiménez} \tag*{}$

$ \text{Agosto 2023 - Versión 1.0} \tag*{} $

En esta publicación presentamos el método analítico paso a paso para encontrar los parámetros $\alpha$ y $\beta$ en el algoritmo __Alpha-Max-Plus-Beta-Min__ utilizado para calcular el módulo de un número complejo $z = I + iQ$ de forma rápida en un hardware con una pequeña pérdida de precisión por medio de la siguiente aproximación lineal $^{\mathbf{[1]}}$:

$\displaystyle |z| = \sqrt{I^2 + Q^2} \approx \alpha \cdot \underbrace{\max(|I|,|Q|)}_{=x} + \beta \cdot \underbrace{\min(|I|,|Q|)}_{=y} \tag{1}$

Aquí $i^2:=-1$ denota la definición de la unidad imaginaria, aunque en los textos de los ingenieros eléctricos y electrónicos prefieren usar la letra $j$ como la unidad imaginaria en lugar de $i$. 

El cómputo de esta aproximación en software produce un resultado 3-4 veces más rápido (en lenguaje C) que el cálculo 100% exacto del módulo con la raíz cuadrada. Sin embargo, en un hardware DSP (Digital Signal Processing) esta aproximación es muy rápida. Se puede hacer el cómputo en un hardware más simple sacrificando un poco más la precisión. Tal como dijo Simon Graffe en su respuesta a esta misma pregunta $^{\mathbf{[2]}}$:

> al restringir las constantes $\alpha$ y $\beta$ a recíprocos de potencias enteras de 2, la transformación se presta bien a la implementación en aritmética de enteros binarios.

A continuación se muestra un ejemplo de implementación en hardware de este algoritmo para $\alpha = 15/16$, $\beta = 15/32$

<img src="images/hardware_implementation_sample.png" width="600" height="600" style="margin:auto" />
<p style="text-align: center">
    Figura 1. Implementación en hardware usando $\alpha = 15/16$,  $\beta = 15/32$ $^{\mathbf{[1]}}$.
</p>

Donde las lineas diagonales $\mathbf{\backslash 1}$ en la figura anterior denotan un desplazamiento de 1 bit a la derecha (más conocido en inglés como _hardwired shift to right_) para implementar en hardware la "división entre dos", y de forma parecida, $\mathbf{\backslash 4}$ indica un desplazamiento de 4 bits hacia la derecha para realizar una "división entre $2^4 = 16$" $^{\mathbf{[1]}}$. Estos desplazamientos de bits por hardware son mucho más rápidos que los desplazamientos con software, ya que no requieren la ejecución de código, y esto lo hace ideales para aplicaciones donde el rendimiento es crítico como en comunicaciones de alta velocidad o procesamiento de imágenes $^{\mathbf{[3]}}$. La línea de control $|I|>|Q|$ es verdadera cuando $|I|$ es mayor que $|Q|$ (bloque comparador). 

Sin embargo, encontrar los valores de los parámetros es un problema puramente matemático, aquí no nos enfocaremos en explicar hardware DSP, para eso están los libros como $\mathbf{[1]}$ y $\mathbf{[3]}$. Me pareció interesante compartirlo, ya que no es tan fácil encontrar esta demostración con el procedimiento matemático detallado en algún artículo o en la web. En la Wikipedia sólo aparece el resultado con el valor del error pico y en los libros de señales digitales no aparece la demostración simbólica. Además, se comparan los valores obtenidos con los numéricos usando la librería ```scipy``` de Python y la herramienta Solver de Excel con macros de VBA (especialmente para aquellos que prefieren trabajar con hojas de cálculo). Si estás aprendiendo integrales, cálculo multivariable y optimización, esta publicación puede que sea útil para ti. También, para todas esas personas curiosas que están aprendiendo. Aunque desafortunadamente lo que se presentará aquí es más cercano a "ingeniería" que de rigor matemático. Si te resulta un poco largo, puedes marcarlo y leerlo luego con calma. 


__Palabras clave:__ módulo de un número complejo, distancia cuadrática entre dos funciones, error pico, optimización, algoritmo alpha-max-plus-beta-min.


__1. Introducción:__

Aquí presentaremos brevemente algunas definiciones básicas antes de resolver este problema analíticamente.

__Distancia cuadrática entre dos funciones y error RMS:__ Sean $n \ge 1$, $m \ge 0$ enteros. Sean $f,g:D \rightarrow \mathbb{R}$ dos funciones diferenciables en el dominio $D \subset \mathbb{R}^n$, en el que la función $g$ tiene $m$ parámetros ajustables denotados por el vector $\mathbf{k} \in K \subset \mathbb{R}^m$. Se define "distancia cuadrática entre dos funciones $f$ y $g$ en el dominio $D$" al valor de la siguiente integral $^{\mathbf{[3]}}$:

$\displaystyle \Psi (\mathbf{k}) := \int_D \left ( f(\mathbf{x}) - g(\mathbf{x}, \mathbf{k})   \right )^2 d \lambda(\mathbf{x}) \tag{2}$

donde $d \lambda(\mathbf{x})= dx_1 dx_2 \ldots dx_n$, siendo $\lambda(D)$ una medida de Lebesgue $^{\mathbf{[4]}}$ (longitud, área, volumen, etc); en la sección 4 de [mi otra respuesta en Quora]( https://es.quora.com/Es-correcta-esta-afirmaci%C3%B3n-displaystyle-int_0-1-int_0-2-int_0-3-ldots-int_0-n-sum_-k-1-n-x_k-dx_1dx_2-ldots-dx_n-frac-n-n-1-4/answer/Luis-Jim%C3%A9nez-127?ch=15&oid=1477743685004910&share=6aeb29f0&srid=uS8BBs&target_type=answer) explico qué es una medida de Lebesgue. Para encontrar los valores de los parámetros $\mathbf{k}$ que minimizan la distancia dada en (2), se encuentra el gradiente y se iguala a cero:

$\nabla \Psi (\mathbf{k}) = \mathbf{0} \tag{3}$

Frecuentemente se utiliza el error RMS $^{\mathbf{[3]}}$ (_Root Mean Square error_) que es la raíz cuadrada del promedio de las desviaciones cuadráticas, es decir

$\displaystyle \operatorname{RMS}(\mathbf{k}) := \sqrt{\frac{\displaystyle \int_D \left ( f(\mathbf{x}) - g(\mathbf{x}, \mathbf{k})   \right )^2 d \lambda(\mathbf{x}) }{\lambda(D)}}  \tag{4}$

__Error promedio:__ Es el promedio de las desviaciones entre las dos funciones en el dominio $D$:

$\displaystyle \varepsilon_{\text{avg}}(\mathbf{k}) := \frac{\displaystyle \int_D \left ( f(\mathbf{x}) - g(\mathbf{x}, \mathbf{k})   \right ) d \lambda(\mathbf{x}) }{\lambda(D)}  \tag{5}$

__Error absoluto y error pico:__ La desviación absoluta $\varepsilon_{\text{abs}}(\mathbf{x},\mathbf{k})$ entre dos funciones $f,g$ está definida por

$\displaystyle \varepsilon_{\text{abs}}(\mathbf{x},\mathbf{k}) := \left |  f(\mathbf{x}) - g(\mathbf{x}, \mathbf{k})  \right | \tag{6}$

El __error pico__ ($\varepsilon_{\text{max}}$) es el valor máximo de $\varepsilon_{\text{abs}}(\mathbf{x},\mathbf{k})$ en el dominio $\mathbf{x} \in D$ y $\mathbf{k} \in K$:

$\displaystyle \varepsilon_{\text{max}} := \max_{\substack{\mathbf{x} \in D \\ \mathbf{k} \in K}} \varepsilon_{\text{abs}}(\mathbf{x},\mathbf{k}) \tag{7}$

__Decibeles:__ Es una medida comunmente utilizada en análisis de señales y en la teoría de control $^{\mathbf{[2]}}$ para expresar un valor positivo $M$ que varía mucho en órdenes de magnitud por lo que es más conveniente representarlo en escala logarítmica:

$M_{\text{dB}} := 20 \log_{10} M \tag{8}$

Ya presentadas estas definiciones, ahora si respondamos a la pregunta original.


__2. Minimización de la distancia cuadrática o minimización del RMS:__

Sean $x = \operatorname{Max} = \max(|I|,|Q|)$,   $y = \operatorname{Min} = \min(|I|,|Q|)$, los valores máximo y mínimo, respectivamente, de los valores absolutos de las partes reales e imaginarias de $z = I + iQ$. Dado que $x$ y $y$ son positivos ya que son valores absolutos ($x,y \ge 0$) y siempre $x \ge y$, entonces el dominio del par $(x,y)$ en el plano cartesiano viene dado por

$D_R = \left \{ (x,y): x^2 + y^2 \le R^2, ~~0 \le y \le x,~~ 0 \le R < \infty \right \} \tag{9}$

y esta región del plano se grafica en la siguiente figura

<img src="images/circular_section.png" width="250" height="250" style="margin:auto" />
<p style="text-align: center">
    Figura 2. Dominio $D_R$.
</p>

Deseamos minimizar la distancia entre $f(x,y) = \sqrt{x^2 + y^2}$ y la aproximación lineal $g(x,y,\alpha,\beta) = \alpha x + \beta y$, por lo que definimos la siguiente función que representa la suma de todas las desviaciones cuadráticas en el dominio $D_R \subset \mathbb{R}^2$:

$ \displaystyle \Psi(\alpha, \beta) = \iint_{D_R} \left ( \sqrt{x^2 + y^2} - \alpha x - \beta y \right )^2 \, dxdy \tag{10}$

Es más fácil resolver la integral (10) en coordenadas polares, por lo que sustituimos $x = r \cos \theta,~ y = r \sin \theta$. No debemos olvidar el jacobiano en coordendas polares que es un factor $r$ multiplicando el integrando, por lo que 

$ \displaystyle \Psi(\alpha, \beta) = \iint_{D_R^*} \left ( r - \alpha r \cos \theta - \beta r \sin \theta \right )^2 \, rdrd \theta \tag{11a}$

donde el dominio de integración se transforma en 

$D_R^* = \left \{ (r, \theta): 0 \le r \le R, ~~0 \le \theta \le \dfrac{\pi}{4},~~ 0 \le R < \infty \right \} \tag{11b}$

sustituyendo los límites de integración, la Ecuación (11a) queda

$ \displaystyle \Psi(\alpha, \beta) = \int_0^R r^3 \, dr \int_0^{\pi/4} \left ( 1 - \alpha  \cos \theta - \beta  \sin \theta \right )^2 \, d \theta \tag{11c}$

integrando (11c) se tiene que 

$ \displaystyle \Psi(\alpha, \beta) = \frac{R^4}{4} \int_0^{\pi/4} \left ( 1 - \alpha  \cos \theta - \beta  \sin \theta \right )^2 \, d \theta = \\ \displaystyle \frac{R^4}{4} \int_0^{\pi/4} \left ( 1 - 2 \alpha  \cos \theta - 2 \beta  \sin \theta +\alpha \beta \sin(2 \theta) + \alpha^2 \cos^2 \theta + \beta^2 \sin^2 \theta \right ) \, d \theta = \\ \displaystyle \frac{R^4}{4}  \left ( \frac{\pi}{4} -  \alpha \sqrt{2} -  \beta (2 - \sqrt{2}) + \frac{1}{2} \alpha \beta  + \alpha^2 \left ( \frac{\pi}{8} + \frac{1}{4}   \right ) + \beta^2 \left ( \frac{\pi}{8} - \frac{1}{4}   \right ) \right ) \tag{11d}$

Las integrales definidas sobre las funciones coseno, seno y sus cuadrados en el intervalo $[0,\pi/4]$ se muestran en el Apéndice A. Para encontrar los valores de $\alpha$ y $\beta$ que minimizan $\Psi( \alpha, \beta)$, se toma gradiente y se iguala a cero, $\nabla \Psi (\alpha, \beta) = \mathbf{0}$

$\left \{ \begin{array}{l} \displaystyle \dfrac{\partial \Psi}{\partial \alpha} = \dfrac{R^4}{4} \left ( 0 - \sqrt{2} - 0 + \dfrac{1}{2} \beta + \alpha \left ( \frac{\pi}{4} + \frac{1}{2}   \right ) \right  ) = 0 \\ \displaystyle \dfrac{\partial \Psi}{\partial \beta} = \dfrac{R^4}{4} \left ( 0 - 0 - (2 - \sqrt{2}) + \dfrac{1}{2} \alpha + \beta \left ( \frac{\pi}{4} - \frac{1}{2}   \right ) \right  ) = 0 \end{array} \right . \tag{12a} $

Tal como se muestra en la expresión anterior (12a), esta optimización de los parámetros $(\alpha, \beta)$ no depende del valor del radio $R$ por lo que estos valores no dependen del módulo del número complejo. Simplificando (12a), podemos escribirlo como el siguiente sistema de ecuaciones lineal:

$ \left \{ \begin{array}{l} \displaystyle  \left ( \frac{\pi}{4} + \frac{1}{2}   \right ) \alpha + \frac{1}{2} \beta = \sqrt{2} \\ \displaystyle \frac{1}{2} \alpha + \left ( \frac{\pi}{4} - \frac{1}{2}   \right ) \beta = 2 - \sqrt{2} \end{array} \right. \tag{12b} $

Al resolver el sistema de ecuaciones, se obtienen los siguientes valores de $\alpha$ y $\beta$:

$ \left \{ \begin{array}{l} \alpha = 0.9475436362 \ldots \\ \beta = 0.3924854252 \ldots \end{array} \right. \tag{12c} $

La matriz Hessiana de $\Psi(\alpha, \beta)$ para comprobar si los puntos obtenidos corresponden a un mínimo relativo, un máximo relativo o un punto de silla viene dada por

$ \mathbf{H}(\alpha,\beta) = \begin{bmatrix}  \dfrac{\partial^2 \Psi}{\partial \alpha ^2} & \dfrac{\partial^2 \Psi}{\partial \beta \partial \alpha} \\  \dfrac{\partial^2 \Psi}{\partial \alpha \partial \beta} & \dfrac{\partial^2 \Psi}{\partial \beta ^2} \end{bmatrix} = \begin{bmatrix} \displaystyle \left ( \frac{\pi}{4} + \frac{1}{2}   \right ) \frac{R^4}{4} & \displaystyle \frac{R^4}{8} \\ \displaystyle \frac{R^4}{8} & \displaystyle \left ( \frac{\pi}{4} - \frac{1}{2}   \right ) \frac{R^4}{4} \end{bmatrix} \tag{13a} $

cuyo determinante viene dado por

$ \operatorname{det} \left ( \mathbf{H} (\alpha,\beta) \right ) = \dfrac{R^8}{16} \left ( \left ( \dfrac{\pi}{4}  \right )^2 - \left ( \dfrac{1}{2}  \right )^2 - \left ( \dfrac{1}{2}  \right )^2 \right ) = \\ \displaystyle \dfrac{R^8}{16} \left (  \dfrac{\pi^2}{16}  - \dfrac{1}{2} \right ) >  \dfrac{R^8}{16} \left (  \dfrac{3^2}{16}  - \dfrac{1}{2} \right ) = \dfrac{R^8}{16} \cdot \left (  \dfrac{1}{16} \right )  > 0 \tag{13b}$

Este determinante es positivo y la segunda derivada $\partial^2 \Psi / \partial \alpha ^2 > 0$, entonces el único punto estacionario obtenido en (12c) es un mínimo relativo. Dado que necesariamente la aproximación lineal del módulo debe ser un valor positivo, entonces se tiene la siguiente restricción para $\alpha$ y $\beta$:

$\alpha x + \beta y \ge 0 ~~\Rightarrow ~~ \alpha r \cos \theta + \beta r \sin \theta \ge 0  \tag{14}$

sin embargo la desigualdad (14) siempre se cumple para $\alpha,\beta \ge 0$ ya que $r \ge 0$, además de que el coseno y seno del ángulo toma valores positivos en el intervalo $0 \le \theta \le \pi/4 $. 


__3. Minimización del RMS con la restricción de que__ $\alpha=1$:

En este caso, se fija el valor de $\alpha$ en 1 en la función dada en (11d), se deriva y se iguala a cero para obtener el valor de $\beta$ que minimiza $\Psi(1, \beta)$:

$\displaystyle \dfrac{\partial \Psi}{\partial \beta}(1,\beta) = \dfrac{R^4}{4} \left ( - (2 - \sqrt{2}) + \dfrac{1}{2} \cdot 1 + \beta \left ( \frac{\pi}{4} - \frac{1}{2}   \right ) \right  ) = 0 ~~ \Rightarrow ~~  \beta = \frac{ \dfrac{3}{2} - \sqrt{2}  }{\dfrac{\pi}{4} - \dfrac{1}{2}} \tag{15a}$

$ \left \{ \begin{array}{l} \alpha = 1 \\ \beta = 0.3005851075 \ldots  \end{array}  \right . \tag{15b} $


__4. Obtención de los parámetros__ $\alpha$ __y__ $\beta$ __minimizando el RMS con la restricción de que el error promedio es idénticamente cero:__

El error promedio $\varepsilon_{\text{avg}}(\alpha, \beta)$ en el dominio $D_R$ viene dado por

$ \displaystyle \varepsilon_{\text{avg}}(\alpha, \beta) = \frac{\displaystyle \iint_{D_R} \left ( \sqrt{x^2 + y^2} - \alpha x - \beta y \right ) \, dxdy}{\displaystyle \iint_{D_R} dxdy}  \tag{16a}$

cambiando a coordenadas polares tal como se hizo con la integral (11a):

$ \displaystyle \varepsilon_{\text{avg}}(\alpha, \beta) = \frac{\displaystyle \iint_{D_R^*} \left ( r - \alpha r \cos \theta - \beta r \sin \theta \right ) \, rdrd \theta}{\displaystyle \frac{\pi R^2}{8}} = \\ \displaystyle \frac{8}{\pi R^2} \int_0^R r^2 \, dr \int_0^{\pi/4} (1 - \alpha cos \theta - \beta \sin \theta ) \, d \theta = \\ \displaystyle  \frac{8}{\pi R^2} \cdot \frac{R^3}{3} \left (   \frac{\pi}{4} - \frac{1}{\sqrt{2}} \alpha - \left ( 1 - \frac{1}{\sqrt{2}} \right ) \beta   \right )     \tag{16b}$


Minimizando la función $\Psi(\alpha, \beta)$ sujeto a $\varepsilon_{\text{avg}}(\alpha, \beta) = 0$ mediante el método de multiplicadores de Lagrange, donde los gradientes de ambas funciones deben ser paralelos, $\nabla \Psi \parallel \nabla \varepsilon_{\text{avg}}$, entonces

$\nabla \Psi(\alpha, \beta) = \mu \nabla \varepsilon_{\text{avg}} (\alpha, \beta) ~~ \Rightarrow ~~ \\ \displaystyle  \frac{R^4}{4} \left ( - \sqrt{2} + \dfrac{1}{2} \beta + \alpha \left ( \frac{\pi}{4} + \frac{1}{2}   \right ), - (2 - \sqrt{2}) + \dfrac{1}{2} \alpha + \beta \left ( \frac{\pi}{4} - \frac{1}{2}   \right ) \right  ) = \mu \cdot \dfrac{8R}{3\pi} \left ( -\frac{1}{\sqrt{2}}, \frac{1}{\sqrt{2}} -1  \right ) \tag{17a}$

definiendo la constante $\eta = 32 \mu/(3 \pi R^3)$:

$ \left \{ \begin{array}{l} \displaystyle  \left ( \frac{\pi}{4} + \frac{1}{2}   \right ) \alpha + \frac{1}{2} \beta + \frac{1}{\sqrt{2}} \eta  = \sqrt{2} \\ \displaystyle \frac{1}{2} \alpha + \left ( \frac{\pi}{4} - \frac{1}{2}   \right ) \beta  + \left ( 1 - \frac{1}{\sqrt{2}}   \right ) \eta = 2 - \sqrt{2} \\ \displaystyle \frac{1}{\sqrt{2}} \alpha + \left ( 1 - \frac{1}{\sqrt{2}}   \right ) \beta = \frac{\pi}{4}   \end{array} \right. \tag{17b} $

Resolviendo el sistema de ecuaciones lineal (17b) se obtienen los siguientes valores de $\alpha$ y $\beta$:

$ \left \{ \begin{array}{l} \alpha = 0.9480594489 \ldots \\ \beta = 0.3926990817 \ldots \end{array}  \right . \tag{17c} $


__5. Minimización del error pico:__

El error absoluto en función del radio $r$ y del ángulo $\theta$ viene dado por

$ \varepsilon_{\text{abs}}(r, \theta, \alpha, \beta) = r \left | 1 - \alpha  \cos \theta - \beta  \sin \theta \right | \tag{18}$

Se desea encontrar el error pico o máximo en el intervalo $\theta \in [0,\pi/4]$. Este problema es equivalente al de encontrar el valor máximo del cuadrado de $ \varepsilon_{\text{abs}}(r, \theta, \alpha, \beta)$ que es una función diferenciable, para encontrar el valor de $\theta$ tal que esta función tenga un máximo local:

$  \dfrac{\partial }{\partial \theta} \left ( 1 - \alpha  \cos \theta - \beta  \sin \theta \right )^2 = 2( 1 - \alpha  \cos \theta - \beta  \sin \theta  )(\alpha  \sin \theta - \beta  \cos \theta) = 0  \tag{19a}$

El máximo local en el ángulo $\theta_0$ satisface:

$ \displaystyle \tan \theta_0 = \frac{\beta}{\alpha} \tag{19b}$

A partir de (19b) se construye el siguiente triángulo rectángulo que muestra la relación entre $\alpha, \beta$ y $\theta_0$:

<img src="images/triangle.png" width="200" height="200" style="margin:auto" />
<p style="text-align: center">
    Figura 3. Relacion entre $\alpha, \beta \text{  y  } \theta_0$.
</p>

Con el triángulo se tienen las expresiones para $\cos \theta_0$ y $\sin \theta_0$ en términos de $\alpha$ y $\beta$ para el valor de $\theta_0$ donde ocurre el máximo local:

$ \displaystyle \left \{ \begin{array}{l} \displaystyle \cos \theta_0 = \frac{\alpha}{\sqrt{\alpha^2 + \beta^2}} \\ \displaystyle \sin \theta_0 = \frac{\beta}{\sqrt{\alpha^2 + \beta^2}}    \end{array}  \right . \tag{19c}$

Sustituyendo (19c) en (18), el error absoluto donde ocurre este máximo local viene dado por

$\displaystyle \varepsilon_{\text{abs,1}} = r \left | 1 - \alpha  \cdot \frac{\alpha}{\sqrt{\alpha^2 + \beta^2}}   - \beta  \cdot \frac{\beta}{\sqrt{\alpha^2 + \beta^2}}  \right | = r \left |  1 - \sqrt{\alpha^2 + \beta^2}   \right | \tag{20a}$


Para encontrar el máximo absoluto del error absoluto en $\theta \in [0, \pi/4]$, se evalúan también en (18) los errores absolutos en los ángulos extremos $\theta = 0$ y $\theta = \pi/4$:

$\displaystyle \left \{  \begin{array}{l}   \theta = 0 ~~~ \Rightarrow ~~~ \varepsilon_{\text{abs,2}} = r \left | 1 - \alpha  \cos (0) - \beta  \sin (0) \right | ~~~ \Rightarrow ~~~ \varepsilon_{\text{abs,2}} = r | 1 - \alpha |  \\  \displaystyle   \theta = \frac{\pi}{4} ~~~ \Rightarrow ~~~ \varepsilon_{\text{abs,3}} = r \left | 1 - \alpha  \cos \left ( \frac{\pi}{4} \right ) - \beta  \sin \left ( \frac{\pi}{4} \right ) \right | ~~~ \Rightarrow ~~~ \varepsilon_{\text{abs,3}} = r \left | 1 - \frac{\alpha + \beta}{\sqrt{2}} \right |     \end{array}   \right . \tag{20b}$

El error pico es el máximo entre $\varepsilon_{\text{abs,1}}$, $\varepsilon_{\text{abs,2}}$ y $\varepsilon_{\text{abs,3}}$:

$\displaystyle \varepsilon_{\text{max}}(\alpha, \beta) = r \cdot \max \left ( \left |  1 - \sqrt{\alpha^2 + \beta^2}   \right |, | 1 - \alpha |,  \left | 1 - \frac{\alpha + \beta}{\sqrt{2}} \right |       \right ) \tag{20c}$

Alternativamente (20c) se puede expresar en términos del ángulo $\theta_0$:

$\displaystyle \varepsilon_{\text{max}}(\alpha, \theta_0) = r \cdot \max \left ( \left |  1 - \alpha \sec \theta_0   \right |, | 1 - \alpha |,  \left | 1 - \alpha \cdot \frac{1 + \tan \theta_0}{\sqrt{2}} \right |       \right ) \tag{20d}$

Para ilustrar esto de forma más fácil, estos tres errores (sin el valor absoluto), se muestran indicados con flechas rojas verticales en la siguiente figura, en el que se varían $\alpha$ y $\beta$ hasta que se minimice el máximo entre estos errores:

<img src="images/sqrt_error.gif" width="400" height="400" style="margin:auto" />
<p style="text-align: center">
    Figura 4. Animación de la búsqueda de los valores óptimos al minimizar el error pico.
</p>

Para encontrar los valores de $\alpha$ y $\beta$ que minimizan el error pico, se busca el mínimo absoluto de la función $\varepsilon_{\text{max}}(\alpha, \beta)$. Esta función NO ES DIFERENCIABLE debido a la presencia de los valores absolutos y la función máximo que son funciones a trozos, así que no se puede resolver simplemente derivando e igualando a cero ¿pero cómo se pueden resolver analíticamente los valores de $\alpha$ y $\beta$ en este caso? La respuesta es: por ensayo y error o barriendo con todos los casos posibles y buscando el par $(\alpha, \beta)$ que produce el mínimo para (20c) ó (20d). La función a minimizar también se puede reescribir como la siguiente función a trozos:

$\displaystyle \varepsilon_{\text{max}}(\alpha, \beta) = \left \{ \begin{array}{ccc} \varepsilon_{\text{abs,1}}  & \text{si} & \varepsilon_{\text{abs,1}} \ge \varepsilon_{\text{abs,2}}, ~\varepsilon_{\text{abs,1}} \ge \varepsilon_{\text{abs,3}}  \\ \varepsilon_{\text{abs,2}}  & \text{si} & \varepsilon_{\text{abs,2}} \ge \varepsilon_{\text{abs,1}}, ~\varepsilon_{\text{abs,2}} \ge \varepsilon_{\text{abs,3}} \\ \varepsilon_{\text{abs,3}}  & \text{si} & \varepsilon_{\text{abs,3}} \ge \varepsilon_{\text{abs,1}}, ~\varepsilon_{\text{abs,3}} \ge \varepsilon_{\text{abs,2}}  \end{array} \right . \tag{20e}$

El mínimo ocurre en algún punto en los bordes entre subfunciones. Ya sabemos de acuerdo a los resultados de las secciones anteriores, que el valor de $\alpha$ es ligeramente menor a 1, y $\beta$ es alrededor de $\sim 0.3-0.4$ y con esto podemos conocer a priori el signo del argumento de los valores absolutos y simplificarlos $\left |  1 - \sqrt{\alpha^2 + \beta^2}   \right | = \sqrt{\alpha^2 + \beta^2} - 1 $,  $| 1 - \alpha | = 1 - \alpha~~$    y    $~~\left | 1 - (\alpha + \beta)/\sqrt{2} \right | = 1 - (\alpha + \beta)/ \sqrt{2}$. Usamos mejor la versión (20d) con $\beta = \alpha \tan \theta_0$, buscamos el punto de intersección entre los tres bordes, resolviendo el sistema de ecuaciones:

$\displaystyle \left \{ \begin{array}{c}  1-\alpha = \alpha \sec \theta_0 - 1 \\ \displaystyle 1 - \alpha = 1 - \alpha \cdot \frac{1 + \tan \theta_0}{\sqrt{2}}   \end{array} \right . \tag{21}$

despejando $\alpha$ de la primera ecuación de (21):

$\displaystyle 1-\alpha = \alpha \sec \theta_0 - 1 ~~~ \Rightarrow ~~~ \alpha = \frac{2}{1 + \sec \theta_0} = \frac{2 \cos \theta_0}{1 + \cos \theta_0} \tag{22a}$

sabiendo que $\beta = \alpha \tan \theta_0$:

$\displaystyle \beta = \frac{2 \cos \theta_0}{1 + \cos \theta_0} \cdot \tan \theta_0  = \frac{2 \sin \theta_0}{1 + \cos \theta_0} \tag{22b}$

Sólo falta conseguir el valor del ángulo $\theta_0$, resolviendo la segunda ecuación de (21):

$\displaystyle 1 - \alpha = 1 - \alpha \cdot \frac{1 + \tan \theta_0}{\sqrt{2}} ~~~ \Rightarrow ~~~ 1 = \frac{1 + \tan \theta_0}{\sqrt{2}} ~~~ \Rightarrow ~~~ \tan \theta_0 = \sqrt{2} - 1 \tag*{}$

$\tan^2 \theta_0 = (\sqrt{2} - 1)^2 = 3-2 \sqrt{2} \tag{22c}$

usando las identidades trigonométricas $\cos^2 \theta_0 = \left ( 1 + \cos(2 \theta_0) \right )/2$, $\sin^2 \theta_0 = \left ( 1 - \cos(2 \theta_0) \right )/2$. Sean $t = \cos (2 \theta_0)$  y  $u = 3 - 2 \sqrt{2}$:


$\displaystyle \frac{1 - \overbrace{\cos (2 \theta_0)}^{=t}}{1 + \underbrace{\cos (2 \theta_0)}_{=t}} = \underbrace{3-2 \sqrt{2}}_{=u} ~~~ \Rightarrow ~~~ \frac{1 - t}{1 + t} = u ~~~ \Rightarrow ~~~ t = \frac{1 - u}{1 + u}   \tag*{}$

deshaciendo los cambios de variable y racionalizando:

$\displaystyle \cos(2 \theta_0) = \frac{1 - (3 - 2 \sqrt{2})}{1 + (3 - 2 \sqrt{2})} = \frac{-2 + 2 \sqrt{2}}{4 - 2 \sqrt{2}} = \frac{-1 +  \sqrt{2}}{2 -  \sqrt{2}} = \\ \displaystyle  \frac{(-1 +  \sqrt{2})(2 +  \sqrt{2})}{(2 -  \sqrt{2})(2 +  \sqrt{2})} = \frac{-2 - \sqrt{2} + 2 \sqrt{2} + 2}{2^2 - 2} = \frac{\sqrt{2}}{2} \tag*{}$

por lo tanto

$ \displaystyle \cos(2 \theta_0) = \frac{\sqrt{2}}{2}  ~~~ \Rightarrow ~~~ 2 \theta_0 = \frac{\pi}{4} ~~~ \Rightarrow ~~~ \theta_0 = \frac{\pi}{8} \tag{22d}$

sustituyendo este valor de $\theta_0$ en las ecuaciones (22a) y (22b) se obtienen los valores de $\alpha$ y $\beta$ que minimizan el error pico:

$ \left \{ \begin{array}{l} \alpha = 0.9604338701 \ldots \\ \beta = 0.3978247348 \ldots \end{array}  \right . \tag{22e} $

que son los mismos valores que se encuentran en la página de Wikipedia $^{\mathbf{[5]}}$. Esta instancia es diferente a las anteriores ya que no era simplemente "derivar e igualar a cero para buscar el óptimo" había que tomar en cuenta los errores en los extremos.


__6. Minimización del error pico con la restricción de que__ $\alpha=1$:

Este caso es el mismo que el de la sección anterior, pero sustituyendo $\alpha = 1$ en la Ecuación (20c), y se resuelve de la misma forma: la solución está en el borde entre las subfunciones. Sólo hay que resolver la siguiente ecuación (igualando $\varepsilon_{\text{abs,1}}$  y  $\varepsilon_{\text{abs,3}}$):

$\displaystyle \left |  1 - \sqrt{1^2 + \beta^2}   \right | = \left | 1 - \frac{1 + \beta}{\sqrt{2}} \right |  ~~~ \Rightarrow ~~~  \sqrt{1 + \beta^2} - 1 =  1 - \frac{1 + \beta}{\sqrt{2}}  \tag*{}$

Manipulando algebraicamente la expresión anterior no es difícil probar que se llega a la siguiente ecuación de segundo grado:

$\displaystyle \beta^2 + (4 \sqrt{2} -2) \beta + (-7 + 4 \sqrt{2}) = 0 ~~~ \Rightarrow ~~~ \beta = -(2 \sqrt{2} - 1) \pm 4 \sqrt{1 - \frac{\sqrt{2}}{2}} \tag{23a}$

Se tienen dos raíces para $\beta$, pero sólo se toma la raíz positiva porque necesariamente $\beta$ tiene que ser positivo tal como se explicó anteriormente:

$ \left \{ \begin{array}{l} \alpha = 1 \\ \beta = 0.3363572758 \ldots \end{array}  \right . \tag{23b} $


__7. Comparación de los resultados analíticos con los numéricos:__

Todos los valores anteriores fueron encontrados exclusivamente de manera algebraica, es decir, sin usar métodos iterativos. Ahora vamos a comparar los resultados de las secciones anteriores con los obtenidos mediante métodos numéricos, esto es muy útil para hacer un _double check_ al trabajo. Aquí presentamos la resolución de este problema mediante un programa en Python y alternativamente, también con un sencillo programa en Visual Basic que les va a gustar a aquellas personas que están más familiarizadas en usar Excel. El formato del output que vamos a poner aquí va a ser el mismo que el de la tabla que compartió Simon Graffe en su respuesta $^{\mathbf{[2]}}$, proveniente de un código en lenguaje C de Griffin G. (1999) hospedado en la página web de dspguru.com $^{\mathbf{[6]}}$, cuyas columnas están definidas por el arreglo (name, $\alpha$, $\beta$, Average Error, RMS (dB), Peak Error (dB) ). Para calcular los errores RMS y promedio, Griffin definió estos errores a lo largo de la longitud de arco de una circunferencia de radio unitario en $0 \le \theta \le \pi/4$ en lugar de integrarlos en función del área de la región $D_R$ (como hicimos aquí en esta publicación), sin embargo, los pares de $(\alpha, \beta)$ obtenidos mediante optimización son los mismos ya que son independientes de la constante que multiplica a la función objetivo. El RMS de la referencia $\mathbf{[6]}$ está definido mediante la siguiente Ecuación (24):

$\displaystyle \text{RMS}(\alpha, \beta) = \sqrt{\frac{4}{\pi} \int_0^{\pi/4} (1 - \alpha \cos \theta - \beta \sin \theta)^2 \, d \theta} \tag{24}$


__Programa en Python:__ se utilizaron las librerías ```numpy```, ```scipy``` y ```pandas``` únicamente para generar la tabla. De ```scipy.integrate``` se aplicó la función ```quad()``` para integrar numéricamente las ecuaciones (4) y (5), y de ```scipy.optimize```, la función ```minimize()``` para resolver los problemas de optimización de la minimización del RMS. Para el problema de minimización del RMS con la restricción de que el error promedio es idénticamente cero de la sección 4, se seleccionó el método ```method='trust-constr'``` añadiendo la ecuación de restricción $\varepsilon_{\text{avg}}(\alpha, \beta)=0$ en el parámetro ```constraints```. Por otro lado, dado que la minimización del error pico presentado anteriormente en las secciones 5 y 6 involucran a una función objetivo que no es diferenciable, entonces se utilizó el procedimiento ```differential_evolution``` basado en algoritmos genéticos que son muy útiles para estos casos, y dado que este método es estocástico en su diseño, entonces se resolvió y se promediaron los valores $(\alpha,\beta)$ para un lote de 100 subproblemas para acertar cuatro-cinco cifras significativas. Este código y las tablas con los resultados se muestran más abajo en el Apéndice B.


__Programa en Visual Basic:__ primero se programó la subrutina ```error_function(alpha, beta, type_error, [dB = True])``` para integrar RMS o el error promedio mediante el método de Simpson-3/8 y se utilizó el Add-In (o comunmente llamado "Complemento") Solver de Excel que contiene procedimientos de optimización. Se utilizó la API en VBA del Solver para resolver todos los problemas de las secciones 2-6 en el ```Public Sub minimize_all()```. Para minimizar el RMS se utilizó el motor ```EngineDesc:="GRG Nonlinear"``` (funciones objetivo que son suaves y diferenciables) y para minimizar el error pico se utilizó ```EngineDesc:="Evolutionary"``` basado en algoritmos genéticos, siendo el equivalente a ```differential_evolution``` de ```scipy.optimize```. La tabla en Excel se muestra a continuación:

<img src="images/Excel_solution_alpha_max_plus_beta_min.png" width="600" height="600" style="margin:auto" />
<p style="text-align: center">
    Figura 5. Resultados en Excel con la ayuda de una macro en VBA.
</p>

Para ejecutar este archivo de Excel debes habilitar las macros, tener el complemento Solver activo (esto se hace a través de Opciones de Excel/Complementos) y si quieres leer el código fuente debes tener habilitada la pestaña de desarrollador (Opciones de Excel/Personalizar cinta de opciones). En cuanto a la compatibilidad con versiones de Excel de años anteriores, yo edité el archivo en Excel 2013.


__Enlaces de los archivos:__ Para los interesados, el código fuente de los archivos están hospedados en GitHub:

[Repositorio Github free_codes_for_quora](https://github.com/dardluizuz/free_codes_for_quora)

y en Google Drive:

https://drive.google.com/folderview?id=1qHnlCRA06JXIS4rhyeaLMrLi_tlqbDUs

En caso de haber algún inconveniente en el acceso de estos archivos o algún bug, por favor escríbeme un comentario en esta respuesta, yo estaré pendiente.


__Comparación de los resultados:__ De acuerdo a las tablas ```pandas.DataFrame``` mostradas más abajo, los resultados numéricos de $(\alpha, \beta)$ en Python se desvían un 0.01% con respecto a los valores exactos, así que prácticamente el procedimiento de optimización numérica coincide con los valores teóricos aquí demostrados. También los resultados en la hoja de Excel con la macro programada usando el Solver Add-In se desvían menos de un 0.01% con respecto a los valores exactos, lo que indica que las subrutinas numéricas fueron implementadas exitosamente. Comparando nuestros resultados con los de Griffin $^{\mathbf{[6]}}$, tenemos que coinciden todos los casos a excepción de "min RMS error, $\alpha = 1$" en el que él reporta un valor de $\beta = 0.32326099$ y aquí en esta publicación $\beta = 0.30058511$ (una desviación del 8%), y en "min Peak error, $\alpha$ = 1" con una ligera desviación del 0.1%. 


__8. Conclusiones y palabras finales:__ 

Una vez presentada en esta publicación la demostración analítica de cómo se hallan los valores de $\alpha$ y $\beta$ en el algoritmo alpha-max-plus-beta-min, podemos concluir que los resultados aquí mostrados coinciden con los tabulados en la literatura, replicándolos y verificando que son correctos. Además, los programas editados en Python y en VBA/Excel imprimen resultados acertando 5 cifras significativas. 

Sin más que añadir ¡espero que les haya gustado esta respuesta!


__9. Referencias:__

[1] Lyons, R.G. (2004), Understanding Digital Signal Processing, 2nd Edition, Prentice Hall,  pp. 479-484, ISBN 0-13-108989-7

[2] [Simon Graffe (2023), Respuesta en Quora sobre Alpha-Max-Plus-Beta-Min](https://es.quora.com/C%C3%B3mo-se-determinan-alpha-y-beta-en-el-algoritmo-alpha-max-plus-beta-min/answer/Simon-Graffe?ch=15&oid=1477743674727883&share=44ffbad7&srid=uS8BBs&target_type=answer)

[3] Frerking, M.E. (2003), Digital Signal Processing in Communication Systems, 2nd Edition, Springer Science-Business Media LLC., pp. 548-591, ISBN 978-1-4757-4990-8

[4] Lebesgue, H. (1902), Intégrale, Longueur, Aire. _Annali di Matematica Pura ed Applicata_, __7__, pp. 231–359. doi:10.1007/BF02420592

[5] https://en.m.wikipedia.org/wiki/Alpha_max_plus_beta_min_algorithm 

[6] Griffin, G. (1999), DSP Trick: Magnitude Estimator, URL: https://dspguru.com/dsp/tricks/magnitude-estimator/


__10. Apéndice A. Cálculo de las integrales del coseno y seno en el intervalo__ $\theta \in [0,\pi/4]$:

En esta sección se presentan las integrales definidas sobre las funciones coseno y seno en $0 \le \theta \le \pi/4$ que aparecen en las ecuaciones (11d) y (16b):

$\displaystyle \int_0^{\pi/4} \cos \theta d \theta = \sin \left ( \frac{\pi}{4}  \right ) - \sin \left ( 0 \right ) = \frac{\sqrt{2}}{2} - 0 \tag{A.1}$

$\displaystyle \int_0^{\pi/4} \sin \theta d \theta =  - \left (  \cos \left (  \frac{\pi}{4} \right ) - \cos (0) \right ) = 1 - \frac{\sqrt{2}}{2}    \tag{A.2}$

$\displaystyle \int_0^{\pi/4} \sin (2 \theta) d \theta = \left . -\frac{\cos (2 \theta)}{2} \right |_0^{\pi/4} = \frac{1}{2} - 0   \tag{A.3}$

$\displaystyle \int_0^{\pi/4} \cos^2 \theta d \theta = \int_0^{\pi/4} \frac{1 + \cos (2 \theta)}{2}  d \theta =  \left . \left ( \frac{\theta}{2} + \frac{\sin (2 \theta)}{4} \right ) \right |_{~0}^{~\pi/4} = \frac{\pi}{8} + \frac{1}{4} \tag{A.4}$

$\displaystyle \int_0^{\pi/4} \sin^2 \theta d \theta = \int_0^{\pi/4} \frac{1 - \cos (2 \theta)}{2}  d \theta =  \left . \left ( \frac{\theta}{2} - \frac{\sin (2 \theta)}{4} \right ) \right |_{~0}^{~\pi/4} = \frac{\pi}{8} - \frac{1}{4} \tag{A.5}$


__11. Apéndice B. Código en python para generar los resultados numéricos:__


In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
import pandas as pd
import warnings
warnings.simplefilter('ignore')

def sqrt_error(alpha, beta, type_error, numerical=True):
    
    fun = lambda theta, n: (1 - alpha*np.cos(theta) - beta*np.sin(theta))**n
    integral_value = lambda n: quad(lambda theta: fun(theta, n), 0, np.pi/4)[0]
    
    if type_error=='avg':
        if numerical:
            return 4/np.pi*integral_value(n=1)
        else:
            return 4/np.pi*(np.pi/4 - 1/np.sqrt(2)*alpha - (1 - 1/np.sqrt(2))*beta)
            
    
    elif type_error=='RMS':
        if numerical:
            return np.sqrt(4/np.pi*integral_value(n=2))
        else:
            return np.sqrt(4/np.pi*(np.pi/4 - np.sqrt(2)*alpha - (2 - np.sqrt(2))*beta + \
                           alpha*beta/2 + (np.pi/8 + 1/4)*alpha**2 + (np.pi/8 - 1/4)*beta**2))
            
    else:
        return np.max([np.abs(np.sqrt(alpha**2 + beta**2) - 1),
                       np.abs(1 - alpha),
                       np.abs(1 - (alpha + beta)/np.sqrt(2)) ])
    

def find_alphabeta(objective_function='min_RMS', dB=True, numerical=True):
    
    if objective_function=='min_RMS':
        if numerical:
            opt_results = minimize(lambda p: sqrt_error(p[0], p[1], 'RMS'), [1, 1])
            p = opt_results.x
        
        else:
            A = np.array([[np.pi/4+1/2, 1/2], 
                      [1/2, np.pi/4-1/2]], dtype=float)
            b = np.array([np.sqrt(2), 2-np.sqrt(2)], dtype=float)
            p = np.linalg.solve(A, b)
            
            
    elif objective_function=='min_RMS,avg=0':
        if numerical:
            constraints = {'type': 'eq', 'fun': lambda p: 10**4*sqrt_error(p[0], p[1], type_error='avg')}
            bounds = ((0, None), (0, None))  
            opt_results = minimize(lambda p: sqrt_error(p[0], p[1], 'RMS'), 
                                   [1, 1], method='trust-constr', bounds=bounds, constraints=constraints)
            p = opt_results.x
        
        else:
            A = np.array([[np.pi/4+1/2, 1/2, 1/np.sqrt(2)],
                      [1/2, np.pi/4-1/2,1-1/np.sqrt(2)],
                      [1/np.sqrt(2),1-1/np.sqrt(2),0] 
                       ], dtype=float)
            b = np.array([np.sqrt(2), 2-np.sqrt(2), np.pi/4], dtype=float)
            p = np.linalg.solve(A, b)
            
            
    elif objective_function=='min_RMS,alpha=1':
        if numerical:
            alpha = 1
            opt_results = minimize(lambda beta: sqrt_error(alpha, beta, 'RMS'), 1)
            beta = opt_results.x[0]
            p = (alpha, beta)
            
        else:
            p = [1, (3/2 - np.sqrt(2))/(np.pi/4 - 1/2)]
            
    elif objective_function=='min_peak':
        if numerical:
            bounds = ((0, 10), (0, 10))  
            n_trials = 100
            alpha = np.ones(n_trials, dtype=float)
            beta = np.ones(n_trials, dtype=float)

            for i in range(n_trials):
                opt_results = differential_evolution(lambda p: sqrt_error(p[0], p[1], 'peak'), bounds=bounds)
                p = opt_results.x
                alpha[i], beta[i] = p
            p = (np.mean(alpha), np.mean(beta))
        
        else:
            theta = np.pi/8
            p = [2*np.cos(theta)/(1 + np.cos(theta)), 2*np.sin(theta)/(1 + np.cos(theta))]
            
            
    elif objective_function=='min_peak,alpha=1':
        if numerical:
            bounds = ((0, 10),)
            n_trials = 100
            alpha = 1
            beta = np.ones(n_trials, dtype=float)

            for i in range(n_trials):
                opt_results = differential_evolution(lambda p: sqrt_error(alpha, p, 'peak'), bounds=bounds)
                beta[i] = opt_results.x[0]
            p = (1, np.mean(beta))
        
        else:
            beta = -(2*np.sqrt(2) - 1) + 4*np.sqrt(1 - np.sqrt(2)/2)
            p = (1, beta)
        
    else:
        if type(objective_function) is str: 
            p = eval('(' + objective_function + ')')
        else:
            p = objective_function
        
    alpha, beta = p[0], p[1]
    
    magnitude2dB = lambda x, dB: x*(dB==False) + 20*np.log10(x)*(dB==True) 
    avg_error = sqrt_error(alpha, beta, 'avg', numerical)
    RMS_error = magnitude2dB(sqrt_error(alpha, beta, 'RMS', numerical), dB)
    peak_error = magnitude2dB(sqrt_error(alpha, beta, 'peak', numerical), dB)                                                           
    return ([avg_error, RMS_error, peak_error], p)


def displaytable_alphamax_betamin(name=['min_RMS', 'min_peak','min_RMS,avg=0', 'min_RMS,alpha=1', \
                                        'min_peak,alpha=1','1, 1/2', '1, 1/4', '1, 4/10', '1, 11/32', \
                                        '1, 3/8', '15/16, 15/32', '15/16, 1/2', '31/32, 11/32', \
                                        '31/32, 3/8', '61/64, 3/8', '61/64, 13/32'],
                                  dB=True, numerical=False):
                                                                                                                                         
    error = np.ones((len(name), 3), dtype=float)      
    alpha = np.ones(len(name), dtype=float)
    beta = np.zeros(len(name), dtype=float)

    for j, name_0 in enumerate(name):                                                                      
        error_list, parameters = find_alphabeta(objective_function=name_0, dB=dB, numerical=numerical)
        alpha[j], beta[j] = parameters[0], parameters[1]    
        error[j,:] = error_list

    pd.options.display.float_format = '{:,.8f}'.format
    df = pd.DataFrame({'name':name,
                       'alpha':alpha,
                       'beta':beta,
                       'avg_error':error[:,0],
                       'RMS (dB)':error[:,1],
                       'peak (dB)':error[:,2]})
    return df
    

__Resultados exactos de las secciones 2-6:__

In [2]:
displaytable_alphamax_betamin(numerical=False)

,name,alpha,beta,avg_error,RMS (dB),peak (dB)
0,min_RMS,0.94754364,0.39248543,0.00054407,-32.64343559,-25.60403636
1,min_peak,0.96043387,0.39782473,-0.01305237,-31.37251081,-28.05352855
2,"min_RMS,avg=0",0.94805945,0.39269908,0.00000000,-32.64107207,-25.68986895
3,"min_RMS,alpha=1",1.00000000,0.30058511,-0.01241148,-28.99182818,-21.90055793
4,"min_peak,alpha=1",1.00000000,0.33635728,-0.02575176,-28.25708745,-25.18442624
5,"1, 1/2",1.00000000,0.50000000,-0.08677793,-20.71274459,-18.55985833
6,"1, 1/4",1.00000000,0.25000000,0.00645288,-27.62905168,-18.70211951
7,"1, 4/10",1.00000000,0.40000000,-0.04948561,-25.14704822,-22.26646812
8,"1, 11/32",1.00000000,0.34375000,-0.02850868,-27.95928081,-24.81680478
9,"1, 3/8",1.00000000,0.37500000,-0.04016253,-26.44468155,-23.34976195


__Resultados numéricos:__

In [3]:
displaytable_alphamax_betamin(numerical=True)

,name,alpha,beta,avg_error,RMS (dB),peak (dB)
0,min_RMS,0.94754362,0.39248545,0.00054408,-32.64343559,-25.60403362
1,min_peak,0.96046728,0.39783668,-0.01308690,-31.36691043,-28.04575292
2,"min_RMS,avg=0",0.94804826,0.39272609,0.00000000,-32.64107173,-25.68799813
3,"min_RMS,alpha=1",1.00000000,0.30058496,-0.01241143,-28.99182818,-21.90054629
4,"min_peak,alpha=1",1.00000000,0.33639138,-0.02576448,-28.25579807,-25.18271078
5,"1, 1/2",1.00000000,0.50000000,-0.08677793,-20.71274459,-18.55985833
6,"1, 1/4",1.00000000,0.25000000,0.00645288,-27.62905168,-18.70211951
7,"1, 4/10",1.00000000,0.40000000,-0.04948561,-25.14704822,-22.26646812
8,"1, 11/32",1.00000000,0.34375000,-0.02850868,-27.95928081,-24.81680478
9,"1, 3/8",1.00000000,0.37500000,-0.04016253,-26.44468155,-23.34976195
